<img src = 'fotos/logo_dani.jpeg'>

# Metadatos 

Este dataset contiene información sobre las exportaciones e importaciones de frutas y hortalizas de españa. Las variables son las siguientes:
- Period: periodo en el que se realizó la operación
- Reporter: país/entidad del que se importa o al que se exporta
- Partner: país que se toma como referencia, todos los valores son españa.
- Product: producto de la operación
- flow: si se exporta o se importa
- quantity_in_100kg: cantidad de producto con el que se opera
- value_in_euros: valor de la operación

## Librerías 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Ficheros y rutas de entrada/salida 

In [2]:
dir_in = '../../datos/datos_desarrollo/'
file1_in = 'comercio_exterior_spread.csv'
dir_out = '../../datos/datos_desarrollo'

## Carga de datos 

In [3]:
df_comercioex = pd.read_csv(os.path.join(dir_in, file1_in), sep = ',').drop('Unnamed: 0', axis =  1)

## Preprocesamiento de los datos 

In [4]:
df_comercioex.columns = df_comercioex.columns.str.lower()
df_comercioex.rename(columns = {'�..period': 'period'}, inplace = True)
df_comercioex[df_comercioex.eq(':')] = np.nan

### Tratamiento de strings 

In [5]:
df_comercioex.value_in_euros = df_comercioex.value_in_euros.str.replace(' ', '')
df_comercioex.quantity_in_100kg = df_comercioex.quantity_in_100kg.str.replace(' ', '')
df_comercioex.period = df_comercioex.period.str.lower().str.replace('.', '', regex = False) 

### Cambio de clases 

In [6]:
navidad = df_comercioex[df_comercioex.period.isin(['jan-dec 2018', 'jan-dec 2019', 'jan-dec 2020'])]
df_comercioex_nonav = df_comercioex.drop(navidad.index, axis = 0)
df_comercioex_nonav.period = pd.to_datetime(df_comercioex_nonav.period, format = '%b %Y')

In [7]:
navidad.loc[navidad[navidad.period == ('jan-dec 2018')].index, 'period'] = '25-12-2018'
navidad.loc[navidad[navidad.period == ('jan-dec 2019')].index, 'period'] = '25-12-2019'
navidad.loc[navidad[navidad.period == ('jan-dec 2020')].index, 'period'] = '25-12-2020'
navidad.period = pd.to_datetime(navidad.period, format = '%d-%m-%Y')
df_comercioex = pd.concat([df_comercioex_nonav, navidad], axis = 0)

C:\Users\unito\Anaconda3\envs\no_sklearn\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\unito\Anaconda3\envs\no_sklearn\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\unito\Anaconda3\envs\no_sklearn\lib\site-packages\pandas\core\generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [8]:
variables_num = ['quantity_in_100kg', 'value_in_euros']
df_comercioex[variables_num] = df_comercioex[variables_num].astype(float)
df_comercioex_num = df_comercioex[variables_num]

### Eliminación de columnas 

In [9]:
df_comercioex.drop('partner', axis = 1, inplace = True)

### Normalización del texto 

In [10]:
def normalize(columna):
   
    ''' Normalizar una columna en formato 'pandas.core.series.Series' que contenga clases primarias de formato string.
        Normalizar significa quitar tildes, ponerlos en formato minúscula y quitar cualquier elemento no ascii:
    
    Args:
        arg_1(pandas.core.series.Series): Columa cuyos elementos strings se quiere normalizar.
        
    Returns:
        pandas.core.series.Series: La columna normalizada. '''
    
    columna_norm = columna.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
    return columna_norm

In [11]:
to_normalize = ['reporter', 'product', 'flow']
df_comercioex[to_normalize] = df_comercioex[to_normalize].apply(normalize)

In [12]:
df_comercioex.reporter = df_comercioex.reporter.str.replace(r'\(.*\)', '', regex = True)

In [13]:
df_comercioex.to_csv(os.path.join(dir_out, 'comercioex_clean.csv'), sep = ';', index = False)